# Preparação da base de dados para treinar os classificadores

In [1]:
from src import (
    Folder,
    GetModel,
    SegmentedList,
    segmentation_fails,
    ds2df,
    save_results,
    File2df,
    Best_rf,
    Rf,
    MLP,
    SVM,
    XGBoost,
    KNN,
    GBM,
    List2Best,
    Save_model
    )

from os.path import join
DATA = "Dados"
MODEL = "Models"

In [2]:
#adquindo as imagens e seus respectivos rotulos
#dataBasePath = join(DATA,"dados")
dataBasePath = join(DATA,"changed","rotacionados")
#dataBasePath = join(DATA,"changed","segmentation")
images, labels = Folder(dataBasePath)

## Caso as imagens não estejam segmentadas, podemos segmentalas

In [3]:

segmentation_model = GetModel(join("models","best.pt"))
process_images = SegmentedList(images,segmentation_model,is_resized=False) # Por padrão redimensionar garante menor taxa de falha
#process_images = SegmentedList(images,segmentation_model,is_resized=True)

WARNING ⚠️ 'boxes' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_boxes' instead.

0: 640x640 (no detections), 479.7ms
Speed: 15.3ms preprocess, 479.7ms inference, 9.9ms postprocess per image at shape (1, 3, 640, 640)
WARNING ⚠️ 'boxes' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_boxes' instead.

0: 640x640 (no detections), 349.4ms
Speed: 5.6ms preprocess, 349.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING ⚠️ 'boxes' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_boxes' instead.

0: 640x640 (no detections), 311.8ms
Speed: 0.0ms preprocess, 311.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING ⚠️ 'boxes' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_boxes' instead.

0: 640x640 1 eye_membrane, 275.5ms
Speed: 0.0ms p

### com as imagens processadas, precisamos descartar as imagens que não foram segmentadas e salvar seu nome em um arquivo com as falhas

In [4]:
fails = segmentation_fails(process_images,labels)

print(f"Quantidade: {len(fails)}\nFalhas:",fails)
print("% = ",len(fails)/len(images))

#msg = "falhas-normais-"
#msg = "falhas-redimensionadas-"


Quantidade: 179
Falhas: ['90_640x640img95_3.jpg', '90_640x640img88_5.jpg', '90_640x640img4_4.jpg', '90_640x640img38_3.jpg', '90_640x640img276_2.jpg', '90_640x640img275_2.jpg', '90_640x640img240_2.jpg', '90_640x640img237_2.jpg', '90_640x640img233_2.jpg', '90_640x640img231_3.jpg', '90_640x640img219_4.jpg', '90_640x640img218_3.jpg', '90_640x640img214_3.jpg', '90_640x640img213_2.jpg', '90_640x640img211_3.jpg', '90_640x640img20_4.jpg', '90_640x640img207_3.jpg', '90_640x640img202_3.jpg', '90_640x640img201_3.jpg', '90_640x640img198_3.jpg', '90_640x640img195_1.jpg', '90_640x640img194_2.jpg', '90_640x640img193_3.jpg', '90_640x640img191_3.jpg', '90_640x640img190_3.jpg', '90_640x640img188_3.jpg', '90_640x640img185_5.jpg', '90_640x640img180_3.jpg', '90_640x640img171_3.jpg', '90_640x640img170_2.jpg', '90_640x640img168_3.jpg', '90_640x640img167_4.jpg', '90_640x640img165_3.jpg', '90_640x640img163_4.jpg', '90_640x640img162_4.jpg', '90_640x640img160_4.jpg', '90_640x640img159_3.jpg', '90_640x640img158_3

### Podemos salvar as falhas em um arquivo de texto para vizualizar depois

In [6]:
anotation_fails= join("docs","Falhas","teste.txt")

with open(anotation_fails, "w") as arquivo:
        for fail in fails:
            arquivo.write(fail + "\n")

### fazendo a filtragem das imagens válidas

In [5]:
valid_images = []
valid_labels = []

for i in range(0,len(process_images),1):
    if labels[i] not in fails:
        valid_images.append(process_images[i])
        valid_labels.append(labels[i])
        

print(len(valid_labels),len(valid_images))

790 790


# Extraindo as Caracteristicas de interesse da base de dados

In [6]:
#df = ds2df(process_images,labels)
#df = ds2df(valid_images,valid_labels)
df = ds2df(images,labels)

### Visando evitar repetir o processamento para a mesma base de dados podemos exportar a mesma para um arquivo e apenas ler a partir do arquivo quando quisermos novamente

In [ ]:

save_results(df,DATA,"rotacionadas",".csv")

### Carregando nossos dados a partir de um dataset .csv

In [3]:
# caso não tenhamos nosso DataFrame carregado, podemos carregalo apartir de um arquivo
df = File2df(join(DATA,"rotacionadas.csv"))

#df.drop("Unnamed: 0",axis=1,inplace=True)
#df.columns

.csv


# Treinando os classificadores

In [ ]:
rf = Best_rf(df,join("docs","classification"))


### Vamos pegar a melhor versão de cada modelo

In [4]:
bests_list = []

funcs = [Rf, MLP,SVM,XGBoost,KNN,GBM]
names = ["RF","MLP","SVM","XGBoost","KNN","GBM"]

args = (0.1,0.2,0.3)

for i,func in enumerate(funcs,0):

    for arg in args:
        
        model, accuracy, precision, recall, f1, kappa = func(df,arg)
        bests_list.append((names[i],arg,accuracy, precision, recall, f1, kappa))
    
bests_list

[('RF', 0.1, 98.97, 98.51, 100.0, 99.25, 97.61),
 ('RF', 0.2, 97.94, 96.92, 100.0, 98.44, 95.41),
 ('RF', 0.3, 91.41, 95.88, 90.06, 92.88, 82.08),
 ('MLP', 0.1, 58.76, 100.0, 39.39, 56.52, 29.35),
 ('MLP', 0.2, 72.68, 77.86, 80.95, 79.38, 38.96),
 ('MLP', 0.3, 68.04, 77.16, 69.06, 72.89, 34.26),
 ('SVM', 0.1, 70.1, 76.81, 80.3, 78.52, 29.45),
 ('SVM', 0.2, 70.1, 75.76, 79.37, 77.52, 32.98),
 ('SVM', 0.3, 71.13, 77.09, 76.24, 76.67, 38.83),
 ('XGBoost', 0.1, 100.0, 100.0, 100.0, 100.0, 100.0),
 ('XGBoost', 0.2, 100.0, 100.0, 100.0, 100.0, 100.0),
 ('XGBoost', 0.3, 92.78, 98.19, 90.06, 93.95, 85.05),
 ('KNN', 0.1, 75.26, 85.0, 77.27, 80.95, 45.89),
 ('KNN', 0.2, 72.68, 82.88, 73.02, 77.64, 42.9),
 ('KNN', 0.3, 73.2, 83.66, 70.72, 76.65, 45.71),
 ('GBM', 0.1, 91.75, 89.19, 100.0, 94.29, 79.64),
 ('GBM', 0.2, 94.85, 95.31, 96.83, 96.06, 88.6),
 ('GBM', 0.3, 89.0, 94.61, 87.29, 90.8, 77.18)]

In [6]:
output = join("docs","classification")

In [7]:
#best = List2Best(bests_list,output,f"{len(valid_images)}.csv",True)
best = List2Best(bests_list,output,f"F{len(images)}.csv",True)
best

'XGBoost'

# caso gostemos do resultado e desejemos salvar o modelo

In [ ]:
model = XGBoost(df,0.1)
Save_model(model,join(MODEL,best))

# Avaliando o comparativo dos modelos 

In [ ]:
# caso não tenhamos nosso DataFrame carregado, podemos carregalo apartir de um arquivo
result_models = File2df(join(output,"not rf.csv"))

#result_models.drop("Unnamed: 0",axis=1,inplace=True)
result_models.columns

In [ ]:
result_models

In [ ]:
rotulos = result_models.Modelo.unique()
rotulos

In [ ]:
# Suponha que 'df' seja o DataFrame contendo os resultados dos modelos

# Rótulos dos modelos
model_labels = ["RF","MLP","SVM","XGBoost","KNN","GBM"]

# Lista para armazenar os melhores resultados de cada modelo
best_results = []

# Iterar sobre os rótulos dos modelos
for model_label in model_labels:
    # Filtrar o DataFrame para o modelo atual
    model_df = result_models[result_models['Modelo'] == model_label]
    
    # Encontrar a linha com o melhor resultado para o modelo atual
    best_row = model_df.loc[model_df['Total'].idxmax()]
    
    # Adicionar o resultado à lista de melhores resultados
    best_results.append(best_row)

import pandas as pd
# Criar um DataFrame com os melhores resultados
print(best_results)
best_results_df = pd.DataFrame(best_results)

# Exibir o DataFrame com os melhores resultados
print(best_results_df)
